# Reading the JSON file and extracting the relevant Users/info, Then inserting into the PostgreSQL Table

In [ ]:
#Importing Necessary Liibraries
import pandas as pd
import numpy as np
import psycopg2 #PostgreSQL
import json

In [ ]:
#PostgreSQL Connection
# Connecting to the PostgreSQL server. Replace user, password, and dbname with your credentials
conn = psycopg2.connect(
    dbname="twitter_data",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
conn.autocommit = True

In [ ]:
try:
    cursor = conn.cursor()
    create_table_query = """CREATE TABLE IF NOT EXISTS user_details (
        id_str VARCHAR(255),
        name VARCHAR(255),
        screen_name VARCHAR(255),
        url VARCHAR(255),
        description TEXT,
        verified BOOLEAN,
        followers_count INT,
        friends_count INT,
        favourites_count INT,
        statuses_count INT,
        created_at TIMESTAMP
        );
        """
    cursor.execute(create_table_query)
    print("Table 'user_details' created successfully or already exists.")
except psycopg2.Error as e:
    print("Error creating table:", e)

In [ ]:
def insert_user(user_data, table_name='user_details'):
    """Insert a single user into the specified table."""
    try:
        cursor = conn.cursor()
        
        # Check if user already exists
        select_query = f"SELECT EXISTS(SELECT 1 FROM {table_name} WHERE id_str = %s);"
        cursor.execute(select_query, (user_data['id_str'],))
        user_exists = cursor.fetchone()[0]

        if user_exists:
            print(f"User with ID {user_data['id_str']} already exists in the database. Skipping insertion.")
            return

        # Insert the new user
        columns = ", ".join(user_data.keys())
        values_placeholder = ", ".join(["%s" for _ in user_data.keys()])
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholder});"
        cursor.execute(insert_query, tuple(user_data.values()))
        print(f"User with ID {user_data['id_str']} inserted into '{table_name}' table.")
    except psycopg2.Error as e:
        print("Error inserting user:", e)

def find_user_by_id(user_id, table_name='user_details'):
    """Find a user by their ID in the specified table."""
    try:
        select_query = f"SELECT * FROM {table_name} WHERE id_str = %s;"
        cursor.execute(select_query, (user_id,))
        user = cursor.fetchone()
        if user:
            print("User found:")
            print(user)
        else:
            print("User not found.")
    except psycopg2.Error as e:
        print("Error finding user:", e)


def find_user_by_name(name, table_name='user_details'):
    """Find a user by their name in the specified table."""
    try:
        select_query = f"SELECT * FROM {table_name} WHERE name = %s;"
        cursor.execute(select_query, (name,))
        user = cursor.fetchone()
        if user:
            print("User found:")
            print(user)
        else:
            print("User not found.")
    except psycopg2.Error as e:
        print("Error finding user:", e)

def find_user_by_screen_name(screen_name, table_name='user_details'):
    """Find a user by their screen name in the specified table."""
    try:
        select_query = f"SELECT * FROM {table_name} WHERE screen_name = %s;"
        cursor.execute(select_query, (screen_name,))
        user = cursor.fetchone()
        if user:
            print("User found:")
            print(user)
        else:
            print("User not found.")
    except psycopg2.Error as e:
        print("Error finding user:", e)

def delete_user_by_id(user_id, table_name='user_details'):
    """Delete a user by their ID from the specified table."""
    try:
        delete_query = f"DELETE FROM {table_name} WHERE id_str = %s;"
        cursor.execute(delete_query, (user_id,))
        print("User deleted successfully.")
    except psycopg2.Error as e:
        print("Error deleting user:", e)


def extract_user_info(data):
    return {
        'id_str': data['user']['id_str'],
        'Name': data['user']['name'],
        'Screen_name': data['user']['screen_name'],
        'url' : data['user']['url'],
        'description': data['user']['description'],
        'verified': data['user']['verified'],
        'followers_count': data['user']['followers_count'],
        'friends_count': data['user']['friends_count'],
        'favourites_count': data['user']['favourites_count'],
        'statuses_count': data['user']['statuses_count'],
        'created_at': data['user']['created_at']
    }

In [ ]:
def process_users(file_path):
    """Process users data from a JSON file and insert them into the MongoDB collection."""
    with open(file_path, "r") as read_file:
        for line_number,line in enumerate(read_file,start=1):
            if line_number%2 !=0:
                data = json.loads(line)
                user1 = extract_user_info(data)
                insert_user(user1)
                if 'retweeted_status' in data:
                    user2 = extract_user_info(data['retweeted_status'])
                    insert_user(user2)
                if 'quoted_status' in data:
                    user3 = extract_user_info(data['quoted_status'])
                    insert_user(user3)


In [ ]:
process_users('corona-out-2.json')

In [ ]:
process_users('corona-out-3.json')